In [2]:
import joblib
import re
import string
from bs4 import BeautifulSoup as bs
import numpy as np
import pandas as pd

import nltk
from nltk.corpus import stopwords  #stopwords
from nltk import word_tokenize,sent_tokenize # tokenizing
from nltk.stem import PorterStemmer,LancasterStemmer  # using the Porter Stemmer and Lancaster Stemmer and others
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer  # lammatizer from WordNet

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, cohen_kappa_score, f1_score, classification_report
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.naive_bayes import MultinomialNB

import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

import warnings
warnings.filterwarnings("ignore") #, category=np.VisibleDeprecationWarning)

In [3]:
df = pd.read_csv('../datasets/current_train_data.csv')
df.head()

,time_spent_ms,raw_text,labels
0,100.0,"BIG LOADS, BRUNETTE, BIG DICKS, CUMSHOTS, PORN...",0
1,100.0,Crushed By Asian Ass MP4.mp4,0
2,100.0,Karol Arrives home and meet's Thatty on the so...,0
3,200.0,Karol Arrives home and meet's Thatty on the so...,0
4,100.0,Robbers Revenge WMV.wmv,0


In [4]:
# function to clean and pre-process the text.
def clean_text(text):  
    
    # 1. Removing html tags
    text = bs(text,"lxml").get_text()
    
    # 2. Retaining only alphabets.
    text = re.sub("[^a-zA-Z]"," ", text)
    
    # 3. Converting to lower case and splitting
    word_tokens = text.lower().split()
    
    # 4. Remove stopwords
    le = WordNetLemmatizer()
    stop_words = set(stopwords.words("english")+ ['14000kbps', 'november', '1080p', 'email', 
                                                 '4k', 'mp4', 'error', '404', '2022', 'hd'])     
    word_tokens = [le.lemmatize(w) for w in word_tokens if not w in stop_words]
    
    cleaned_review = " ".join(word_tokens)
    return cleaned_review

In [5]:
df["text"] = df.raw_text.map(clean_text)

In [6]:
df_train, df_test = train_test_split(df, test_size=0.20, stratify=df.labels, shuffle=True)

In [7]:
vec = CountVectorizer(
    ngram_range=(1, 3), 
#     stop_words="english",
)

X_train = vec.fit_transform(df_train.text)
X_test = vec.transform(df_test.text)

y_train = df_train.labels
y_test = df_test.labels

In [32]:
from sklearn.model_selection import GridSearchCV
param={'alpha': [1e-6, 1e-5]}
scoring = {"AUC": "roc_auc", "Accuracy": 'accuracy', 'F1': 'f1_macro'}

nb = MultinomialNB()
clf=GridSearchCV(nb, param, scoring=scoring, cv=5, refit='AUC', return_train_score=True)  #10

clf.fit(X_train, y_train)
print('Best estimator:', clf.best_estimator_)
      
preds = clf.predict(X_test)
print('Каппа-коэффициент Коэна: ', cohen_kappa_score(y_test, preds))
print(classification_report(y_test, preds))

Best estimator: MultinomialNB(alpha=1e-05)
Каппа-коэффициент Коэна:  0.848880559221093
              precision    recall  f1-score   support

           0       0.99      0.96      0.98     20544
           1       0.82      0.94      0.87      3654

    accuracy                           0.96     24198
   macro avg       0.90      0.95      0.92     24198
weighted avg       0.96      0.96      0.96     24198



In [33]:
new_df = pd.DataFrame(df_test.text)

In [34]:
new_df['predict'] = new_df.apply(lambda x: preds)

In [35]:
new_df.head()

,text,predict
33592,humiliatrix diva miss kendra put thru panty hu...,0
9681,footjob kyra andre clip mp,0
38435,domestic live man mp,0
10894,bg clip wmv,0
91665,slave looser may live locked chastity time tod...,0


In [36]:
new_df.to_csv('../datasets/cat_pred.csv', index=False)

In [37]:
# joblib.dump(nb, "nb.joblib")
joblib.dump(clf, "nb.joblib")
joblib.dump(vec, "vec.joblib")

['vec.joblib']

In [38]:
nb_saved = joblib.load("nb.joblib")
vec_saved = joblib.load("vec.joblib")

In [43]:
%time
# sample_text = ["Space, Stars, Planets and Astronomy!"]
sample_text = ["""jerk instruction asian goddess masturbation encouragement gag talk asian goddess asian princess femdom female domination female supremacy gag talk cleave gag joi jerk instruction masturbation instruction jerk encouragement astrodomina damsel distress rope bondage tape bondage"""]
# sample_text = ["""Well now, buxom powerhouse beauty, Charlie, must have been eating cheese or something equally ‘mind-influencing' shortly before bedtime, ‘cause she's having some mondo creepyass dreamage! All to do with absolute captivity, you know, tight bondage and even tighter gags? As she fitfully starts and stutters, gagged and bound alongside her best friend, blonde saucepot, Hannah, for some bizarre reason, her dreams shift and suddenly she's alone and facedown on the bed, struggling intensely, a mega tight black cloth cleave gag keeping her completely muzzled, and yet still she gag-enthuses passionately, wriggling hard like a fish out of the bowl and totally tied up and helpless... then the dream shifts back again and she's coming around alongside her equally scantily clad friend, who is gagged firmly with tape. As the pair - is this still a dream, or what? - come around and realise they are bound and gagged; cue yet another wondrous moment of sexy wriggling, gag talking and energised attempts to escape the stringent ropes. But, of course, they are too tight! And now all these hapless stunners can do is writhe and moan in their underwear on the bed, pondering who, in Charlie's dream, has done this to them! When their twilit captor finally shows himself, the furious babes protest miraculously into their gags, but he ignores them. In Charlie's dreams he is a dark burglar, intent on molesting them, and relieving them of all their homely possessions. Nice. And, as if to demonstrate, he forces a  new gag around the indignant yapper of Charlie as Hannah struggles on, and unleashes her truly magnificent boobies, bouncing them around for his total pleasure, Then the newly bandanna cleave gagged babe is left again to struggle and strain next to her similarly now boob-revealed chum. Both girls are first tickled and spanked and then shut away to continue their tied up endeavours. After a while of very tasty squirming, they suddenly hatch the utterly unintelligible (and this is precisely why it's so damned sexy) plot to wriggle to their feet and hop away. This they ultimately acheive in one incredibly potent scene but are soon recaptured and both carried jiggling and protesting HARD over the bad guy's shoulder. The ever-resisting hotties are shoved rudely against the bedroom wall and left again to twist and buck, shaking their massive tits all over the place. Next up, as a punishment for their wholly unacceptable escape attempts, the pair are firmly trussed in a hallway and massively gagged with hardcore microfoam tape wraparounds. FEEL them cheeks BULGE, Peeps! Wow, this dream is rapidly descending into the stuff of nightmare - for them. For US, it's the ultimate thrill, right? And, as the amazingly barefoot honnies struggle and emote with total sexiness, we must pinch ourselves every once in a while, surely, to see if we really are awake ourselves.... It doesn't get much more exciting that this, as these two Herculean beauties smash it again and again as the ultimate pair of feisty damsels, and for our complete pleasure.****
# Wow, this incredibly kinky dream of Young Charlie's seems to have been going on a long time, and ratcheting up in intensity as it progresses to boot! Now, the phantom burglar has left the babes bound and gagged in an upstairs hallway where they pluckily communicate with quite scintillating vigour to one another through their massively suppressive microfoam tape wrap muzzles. They cannot say a WORD. Perfecto! What they CAN do is mmmmpppphhhh like crazy and jiggle their massive boobs around while also flexing their sexy bare feet as they struggle and strain, trying urgently to loosen those damned ropes What they actually succeed in doing is tightening them even more, and now things are getting more nightmarish as the ethereal (get the picture?) crook materialises once more and ropes both hotties one by one into complete and ruthlessly close hogties, but not before manhandling and abusing them both with pervy feel ups and spankings, even wrapping his rope tightly around Charlie's massive boobs, squeezing them ultra tightly as she protests indignity. Poor Hannah, roped into her hogtie already, can't even turn around to see what the creep is doing to her friend. The totally incapacitated duo are soon left to struggle passionately, facedown on the rug, wriggling and grunting with the pure exertion of bouncing around on their boobs, their sexy legs drawn up uncomfortably behind them in these wicked-close hogties! No escape, just an urgent wigglefest as the babes try repeatedly - and furiously - to communicate with one another through their overwhelming tape wrap gags. Later, the bad guy is back, and to add to their current woes, repeatedly winds black electricians' tape over the top of their already mondo-gags. This guy must be nuts, but, we are most grateful for his creativity, no doubt. Now the utterly helpless stunners must struggle and roll and twist their way through the final phases of Charlie's incredibly hot bondage dream. We wonder when she'll ever wake up, but would keenly encourage her to continue napping if these are the kind of night-time fancies she experiences. Good for you, Charlie, and thank you, you saucy little beauts for putting on one hell of a lucid (for a dream) action show as you galvanically buck and moan your respective ways to bondage supremacy. Our collective pulse is now through the roof, thanks to you, gals! Happy Effing New Year, Folks. Just  you WAIT to see what thrills 2014 holds here at Borderland Bound!!"""]
# sample_text = ["strap on dildo fucking male strap on amateur forced feminization strap on bondage whipping caning dildo female domination cross dressing spanking humiliation sissy slut big tits MILF blonde BDSM i sissy training dildo blow job        "]
clean_sample_text = clean_text(sample_text[0])
sample_vec = vec_saved.transform(sample_text)
pred = nb_saved.predict(sample_vec)

print("Has bad word" if pred[0] == 1 else "Clear text")


CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 6.91 µs
Has bad word
